In [1]:
# Imports
from sage.groups.perm_gps.permgroup_named import SymmetricGroup
from sage.coding.goppa_code import GoppaCode
from sage.matrix.constructor import matrix
import numpy as np
import itertools
import random

In [2]:
class McEliece:
    """
    Implements the McEliece cryptosystem based on binary Goppa codes. 
    This cryptosystem is known for its security derived from the hardness 
    of decoding a general linear code.
    """

    def __init__(self, n, t):
        """
        Initializes the McEliece cryptosystem with given parameters.

        Parameters:
        - n (int): Length of the Goppa code.
        - t (int): Error-correcting capability of the Goppa code.
        """
        self.dimension = 0
        while self.dimension <= 0:
            m = math.log2(n)
            self.F = GF(2^m, 'a')  # Defines the finite field GF(2^m).
            self.R = PolynomialRing(self.F, 'x')  # Polynomial ring over GF(2^m).
            self.x = self.R.gen()  # Generator of the polynomial ring.
            self.alfa = self.F.gen()  # Generator of the finite field.
            # Generate public and private keys
            self.public_key, self.private_key = self.generate_keys(n, t)
            G, _, _, _, _ = self.private_key
            self.dimension = G.nrows()
        print("Code dimension: ", self.dimension)

        
        
    def generate_keys(self, n, t):
        """
        Generates the public and private keys for the McEliece cryptosystem.

        Parameters:
        - n (int): Length of the code.
        - t (int): Error-correcting capability.

        Returns:
        Tuple of public_key and private_key.
        """
        g = self.R.irreducible_element(t)  # Generates an irreducible Goppa polynomial of degree t.
        L = self.generateL(n, g)  # Support set L, excluding roots of g.
        C = GoppaCode(g, L)  # Constructs the Goppa code.
        G = C.generator_matrix()  # Generator matrix of the Goppa code.
        S = self.generateS(G.nrows())  # Random invertible matrix.
        P = self.generateP(G.ncols())  # Permutation matrix.
        G_prima = S * G * P  # Scrambled generator matrix forming the public key.
        public_key = (G_prima, t)  
        private_key = (G, S, P, g, L)  
        return public_key, private_key
    
    def generateL(self, n, g):
        """
        Generates a set L for the Goppa code, excluding roots of the Goppa polynomial.

        Parameters:
        - n (int): The desired number of elements in L.
        - g (polynomial): The Goppa polynomial.

        Returns:
        A list of elements forming the set L.
        """
        all_elements = self.F.list()  # All elements of the finite field.
        roots_of_g = [alpha for alpha in all_elements if g(alpha) == 0]  # Roots of the Goppa polynomial.
        valid_elements = [alpha for alpha in all_elements if alpha not in roots_of_g]  # Exclude roots of g.
        if len(valid_elements) < n:
            return -1
        return valid_elements[:n]  # Return the first n valid elements.
    
    def generateP(self, n):
        """
        Generates a random permutation matrix P.

        Parameters:
        - n (int): The size of the matrix.

        Returns:
        A permutation matrix P.
        """
        I = np.identity(n, dtype=int)  # Identity matrix.
        np.random.shuffle(I)  # Shuffle rows to create a permutation matrix.
        P = matrix(GF(2), I.tolist())  # Convert to Sage matrix over GF(2).
        return P
    
    def generateS(self, n):
        """
        Generates a random invertible square matrix S over GF(2).

        Parameters:
        - n (int): The size of the matrix (number of rows/columns).

        Returns:
        An invertible matrix S.
        """
        while True:
            S = random_matrix(GF(2), n, n)  # Generate a random matrix.
            if S.rank() == n:  # Check if the matrix is invertible (full rank).
                return S  # Return the invertible matrix.
    
    def encrypt(self, message):
        """
        Encrypts a message using the McEliece public key.

        Parameters:
        - message (vector): The message to be encrypted, represented as a binary vector.

        Returns:
        The encrypted message as a matrix.
        """
        G_prima, t = self.public_key
        error = self.generateErrorVector(G_prima.ncols(), t)  # Generates the error vector.
        ciphertext = matrix(GF(2), [message * G_prima]) + matrix(error)  # Encrypts the message.
        return ciphertext
    
    def generateErrorVector(self, n, t):
        """
        Generates an error vector of a specified length and weight.

        Parameters:
        - n (int): Length of the error vector.
        - t (int): Weight of the error vector (number of errors).

        Returns:
        An error vector.
        """
        error_vector = np.zeros(n, dtype=int)  # Initialize vector of zeros.
        indexes = np.random.choice(n, t, replace=False)  # Randomly select t positions to set to 1.
        error_vector[indexes] = 1
        e = matrix(GF(2), error_vector.tolist())  # Convert to a Sage matrix.
        return e
    
    def decrypt(self, c_prima):
        """
        Decrypts a message using the McEliece private key.

        Parameters:
        - c_prima (matrix): The encrypted message matrix.

        Returns:
        The decrypted message as a binary vector.
        """
        G, S, P, g, L = self.private_key
        c_tilde = c_prima * P.inverse()
        s = self.calculate_syndrome(c_tilde[0], g, L)
        e = self.patterson_algorithm(s, g, L)
        c = c_tilde[0] - e 
        G_right_inverse = G.solve_right(identity_matrix(G.nrows())) 
        mS = c * G_right_inverse
        m = mS * S.inverse()
        return m
    
    def calculate_syndrome(self, received_word, g, L):
        """
        Calculates the syndrome of a received word based on the Goppa polynomial and set L.

        Parameters:
        - received_word (vector): The received word vector.
        - g (polynomial): The Goppa polynomial.
        - L (list): The set of elements forming the support of the Goppa code.

        Returns:
        The syndrome polynomial.
        """
        syndrome = self.R(0)
        for i, coeff in enumerate(received_word):
            if coeff == 0:
                continue
            term = self.R(coeff) * (self.x - L[i]).inverse_mod(g)
            syndrome += term
        syndrome = syndrome.mod(g)
        return syndrome
    
    def patterson_algorithm(self, s, g, L):
        """
        Patterson's algorithm for error correction in Goppa codes.

        Parameters:
        - s (polynomial): The syndrome polynomial.
        - g (polynomial): The Goppa polynomial.
        - L (list): The set of elements forming the support of the Goppa code.

        Returns:
        An error vector indicating the positions of errors.
        """
        # Step 1: find h(x) s.t. s(x)h(x) = 1
        h = s.inverse_mod(g)
        if h == self.x:
            return self.x
        
        # Step 2: find d(x) s.t. d(x)^2 = h(x) + x mod(g(x))
        g0, g1 = self.split(g)
        w = ((g0)* g1.inverse_mod(g)).mod(g)
        T0, T1 = self.split(h + self.x)
        d = (T0 + (w)*(T1)).mod(g)
        
        # Step 3: find a(x) and b(x) s.t. d(x)b(x) = a(x) mod(g(x))
        a, _, b = self.extended_euclidean_algorithm(g, d)
        
        # Step 4: build the error-locating polynomial: sigma(x) = a(x)^2 + xb(x)^2
        sigma = a^2 + self.x*b^2
        
        # Step 5: build the error vector
        error_positions = [i for i, alpha in enumerate(L) if sigma(alpha) == 0]
        e = vector(self.F, [1 if i in error_positions else 0 for i in range(len(L))])
        return e
    
    
    def split(self, p):
        """
        Divides a polynomial into two components containing the square roots of 
        the even-indexed coefficients and the odd-indexed coefficients, respectively.

        Parameters:
        - p (Polynomial): The polynomial to be split, which is an element of a 
                          polynomial ring over a finite field.

        Returns:
        A tuple containing two polynomials (p0, p1) where:
        - p0 contains the square roots of the coefficients at even indices of p.
        - p1 contains the square roots of the coefficients at odd indices of p.

        Both p0 and p1 are elements of the same polynomial ring as p.
        """
        Phi = p.parent()
        p0 = Phi([sqrt(c) for c in p.list()[0::2]])
        p1 = Phi([sqrt(c) for c in p.list()[1::2]])
        return (p0,p1)

    
    def extended_euclidean_algorithm(self, a, b):
        """
        Extended Euclidean algorithm for polynomials, finding polynomials gcd(a,b), u, v 
        such that au + bv = gcd(a,b) and specific degree conditions are met.

        Parameters:
        - a (polynomial): A polynomial over GF(2^m).
        - b (polynomial): Another polynomial over GF(2^m).

        Returns:
        Polynomials gcd(a,b), u and v that satisfy the algorithm's conditions.
        """
        if b.is_zero():
            return a, self.R.one(), self.R.zero()
        s, t = self.R.one(), self.R.zero()
        r0, r1 = a, b
        while True:
            quotient, r2 = r0.quo_rem(r1)
            r0, r1 = r1, r2
            s, t = t, s - t*quotient
            if r1.degree() <= (a.degree() - 1) // 2 and r0.degree() <= a.degree() // 2:
                break
        lead_coeff = r0.leading_coefficient()
        gcd = r0 / lead_coeff
        u = s / lead_coeff
        v = (r0 - a * s) // b
        v /= lead_coeff
        return gcd, u, v
    
    
    def run(self, message=None):
        """
        Demonstrates the complete encryption and decryption process using 
        randomly generated messages or fixed messages.
        
        Parameters:
            message (list of int):  Optional parameter representing a binary message. 
                                    If it is missing, a random message is generated.
        """
        if message is None:
            original_message = self.generate_message()
        else:
            message = vector(GF(2), message)
            assert len(message) == self.dimension, "Provided message length must match the code dimension."
            original_message = message
        encrypted_message = self.encrypt(original_message)
        decrypted_message = self.decrypt(encrypted_message)
        self.print_info(original_message, encrypted_message, decrypted_message)
        
        
    def generate_message(self):
        """
        Generates a random binary message with length equal to the code dimension.
        
        Returns:
            list of int: A random binary message.
        """
        return vector(GF(2), [randint(0, 1) for _ in range(self.dimension)])
    
    
    def print_info(self, original_message, encrypted_message, decrypted_message):
        """
        Prints information about the encryption and decryption process.
        
        Parameters:
            original_message (list of int): The original binary message.
            encrypted_message (list of int): The encrypted binary message.
            decrypted_message (list of int): The decrypted binary message.
        """
        print("Code dimension: ", self.dimension)
        print("Original message: ", original_message)
        print("Encrypted message: ", encrypted_message)
        print("Decrypted message: ", decrypted_message)
        print(original_message == decrypted_message)

In [10]:
# Parameters for the McEliece instance
n = 32  # Code length
t = 2  # Error-correcting capability of the Goppa code

# Creating an instance of McEliece
mc_eliece = McEliece(n, t)

Code dimension:  22


In [11]:
mc_eliece.run()

Code dimension:  22
Original message:  (1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0)
Encrypted message:  [1 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 1 0]
Decrypted message:  (1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0)
True


In [13]:
dimension = 22
message = [randint(0,1) for _ in range(dimension)]
mc_eliece.run(message)

Code dimension:  22
Original message:  (1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1)
Encrypted message:  [1 1 0 0 0 0 1 1 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0]
Decrypted message:  (1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1)
True
